In [2]:
import numpy as np
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.python.keras.preprocessing import image

In [3]:
#Создание генератора
datagen = ImageDataGenerator(rescale=1. / 255)

#Загрузка изображений
train_generator = datagen.flow_from_directory(
    'data/training',
    target_size=(20, 20),
    batch_size=30,
    class_mode='categorical')

val_generator = datagen.flow_from_directory(
    'data/valid',
    target_size=(20, 20),
    batch_size=30,
    class_mode='categorical')

test_generator = datagen.flow_from_directory(
    'data/test',
    target_size=(20, 20),
    batch_size=30,
    class_mode='categorical')



Found 13220 images belonging to 9 classes.
Found 13220 images belonging to 9 classes.
Found 255 images belonging to 9 classes.


In [25]:
#Создание нейронной сети
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='valid', input_shape=(32,32,3), activation='relu', data_format="channels_last"))
model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='valid', activation='relu',data_format="channels_last"))
model.add(MaxPooling2D(pool_size=(2,2), data_format='channels_last'))
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='valid', activation='relu',data_format="channels_last"))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='valid', activation='relu',data_format="channels_last"))
model.add(MaxPooling2D(pool_size=(2,2), data_format='channels_last'))
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='valid', activation='relu',data_format="channels_last"))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='valid', activation='relu',data_format="channels_last"))
model.add(MaxPooling2D(pool_size=(2,2), data_format='channels_last', padding='same'))
model.add(Dropout(0.5))

model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='valid', activation='relu',data_format="channels_last"))
model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='valid', activation='relu',data_format="channels_last"))
model.add(MaxPooling2D(pool_size=(2,2), data_format='channels_last', padding='same'))
model.add(Dropout(0.5))


model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

'''model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(20, 20, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(9))
model.add(Activation('sigmoid'))'''


ValueError: Negative dimension size caused by subtracting 3 from 1 for 'conv2d_73/Conv2D' (op: 'Conv2D') with input shapes: [?,1,1,64], [3,3,64,64].

In [7]:
#Компиляция нейронной сети
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
#Обучеие нейронной сети

# Размер мини-выборки
batch_size = 30
# Количество изображений для обучения
nb_train_samples = 13220
# Количество изображений для проверки
nb_validation_samples = 13220
# Количество изображений для тестирования
nb_test_samples = 255

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=9,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)


Epoch 1/9
440/440 [==============================] - 816s 2s/step - loss: 0.7555 - acc: 0.7406 - val_loss: 0.1413 - val_acc: 0.9684
Epoch 2/9
439/440 [============================>.] - ETA: 0s - loss: 0.1565 - acc: 0.9586

In [52]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)


print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))


img_path = '1q.jpg'
img = image.load_img(img_path, target_size=(40, 40))

# Преобразуем изображением в массив numpy
x = image.img_to_array(img)

# Инвертируем и нормализуем изображение
#x = 255 - x
x /= 255
x = np.expand_dims(x, axis=0)
prediction = model.predict(x)
print(np.argmax(prediction))

 

Аккуратность на тестовых данных: 95.00%
5


In [53]:
# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("model_gen_j.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("model_gen_h.h5")
